In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Checkpoint Preprocesamiento en Text Mining.

En esta práctiva vamos a usar el dataset de mails que ya usamos en la clase de Naive Bayes.

Este dataset (<a href="https://www.kaggle.com/riyadhrazzaq/multinomial-naive-bayes-from-scratch/data?select=spam.csv" traget="_blank">fuente</a>) tiene dos columnas:

* una columna con el cuerpo del mail, 

* y otra con la etiqueta spam / ham según corresponda a un mail que es spam o no respectivamente.

Vamos a preprocesar los textos de los mails usando `CountVectorizer` y `TfidfTransformer` y comparar las representaciones obtenidas.


## Imports

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import seaborn as sns
import matplotlib.pyplot as plt

## Ejercicio 1

Leer los datos del archivo `spam.csv`. 

Mantener sólo las columnas 1 y 2 etiquetarlas como 'target' y 'text'

Ayuda: usar `encoding='iso8859_14'`

In [ ]:
data = pd.read_csv("../Data/spam.csv", encoding='iso8859_14')
data.head()

In [ ]:
data.drop(labels=data.columns[2:],axis=1,inplace=True)
data.columns=['target','text']
data.head()

## Ejercicio 2

Los textos del dataset están en idioma inglés.

Construir una lista de stems de stopwords usando `SnowballStemmer`

Generar una instancia de DataFrame resultado de aplicar CountVectorizer a los textos de los mails, usando como stopwords la lista resultado del paso anterior.


In [ ]:

stopwords_en = stopwords.words('english');

englishStemmer = SnowballStemmer("english")

# si no hacemos esto y usamos directo stopwords_sp, CountVectorizer devuelve un warning
stopwords_en_stem = [englishStemmer.stem(x) for x in stopwords_en]

vectorizer = CountVectorizer(stop_words = stopwords_en_stem, lowercase = True, strip_accents = 'unicode');

vectorizer.fit(data.text);

#print('Vocabulario:\n',vectorizer.vocabulary_) # vocabulario del corpus con la frecuencia de cada término

In [ ]:
countvectorizer_encoding = vectorizer.transform(data.text);

countvectorizer_df = pd.DataFrame(countvectorizer_encoding.todense(), 
             columns = vectorizer.get_feature_names()) # Usamos el método .todense() para ver la matriz completa

countvectorizer_df

## Ejercicio 3

A partir del encoding resultado de CountVectorizer, generar una instancia de DataFrame resultado de aplicar TfidfTransformer.


In [ ]:
tfidf_encoding = TfidfTransformer().fit_transform(countvectorizer_encoding);

tfidf_df = pd.DataFrame(tfidf_encoding.todense(),columns = vectorizer.get_feature_names())

tfidf_df


## Ejercicio 4

Comprobemos que los valores de la representación resultado de CountVectorizer tienen rango de valores disímiles, y distintos a la representación obtenida con TfidfTransformer

Para eso grafiquemos

1. Mediante un scatterplot los valores medios de cada palabra de la representación resultado de CountVectorizer.

2. Mediante un scatterplot los valores medios de cada palabra de la representación resultado de TfidfTransformer.

3. Mediante `errorbar` https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.errorbar.html, 
    
    en el eje x los valores medios de CountVectorizer de cada palabra 
    en el eje y los valores medios de TfidfTransformer de cada palabra 
    como error x los desvíos estándar de CountVectorizer de cada palabra 
    como error y los desvíos estándar de TfidfTransformer de cada palabra 
    
Obervemos que, tal como esperamos, la dispersión es mayor en el eje x (CountVectorizer) que en el eje y (TfidfTransformer)

In [ ]:
countvectorizer_means = [np.mean(countvectorizer_df[col]) for col in countvectorizer_df.columns ]
sns.scatterplot(x = range(len(countvectorizer_means)), y = countvectorizer_means)

In [ ]:
tfidf_means = [np.mean(tfidf_df[col]) for col in tfidf_df.columns ]
sns.scatterplot(x = range(len(tfidf_means)), y = tfidf_means)

In [ ]:
countvectorizer_means = [np.mean(countvectorizer_df[col]) for col in countvectorizer_df.columns]
countvectorizer_sd = [np.std(countvectorizer_df[col]) for col in countvectorizer_df.columns]

tfidf_means = [np.mean(tfidf_df[col]) for col in tfidf_df.columns]
tfidf_sd = [np.std(tfidf_df[col]) for col in tfidf_df.columns]

plt.errorbar(x = countvectorizer_means, y = tfidf_means, yerr = tfidf_sd, xerr = tfidf_sd, linestyle='None', marker='^')
